# 协程：async/await 语法

[点击查看 PEP 492](https://www.python.org/dev/peps/pep-0492/)

### async 表达式

使用如下语法定义原生协程：

```python
async def read_data(db):
    pass
```

- `async def`的函数必定是协程，即使里面不含`await`语句；
- 协程不再抛出`StopIteration`异常，因为抛出的`StopIteration`会被包装成一个`RuntimeError`异常；
- 如果一个协程从未`await`等待就被垃圾收集器销毁了，会引发一个`RuntimeWarning`异常。

### types.coroutine()

使用此装饰器可以将现有的“使用生成器实现的协程”转化为与“原生协程”兼容的形式。

```python
@types.coroutine
def process_data(db):
    data = yield from read_data(db)
    ...
```

### await 表达式

使用`await`表达式获取协程执行的结果：

```python
async def read_data(db):
    data = await db.fetch('SELECT ...')
    ...
```

`await`和`yield from`类似，它挂起`read_data`的执行，直到`db.fetch`执行完毕并返回结果。

`await`仅接受 awaitable 对象（否则TypeError，是下面其中一种：
- 一个原生协程对象；
- 用装饰`types.coroutine`装饰的一个“生成器实现的协程”对象；
- 一个有`__await__`方法（返回一个迭代器）的对象




await语句大多数情况下不需要被圆括号包围。比如：
```python
-await foo() 与 -(await foo()) 等价。
```


但是：
```python
await await coro() 应写成 await (await coro())
await -coro() 应写成 await (-coro())
```

### 异步上下文管理器和 `async with`

异步上下文管理器（asynchronous context manager），可以在它的enter和exit方法里挂起、调用异步代码。
因此有两个魔术方法：`__aenter__`和`__aexit__`，他们返回一个awaitable 对象

```python
class AsyncContextManager:
    async def __aenter__(self):
        await log('entering context')
    
    async def __aexit__(self, exc_type, exc, tb):
        await log('exiting contenxt')
```

并且有一套新的语法：
```python
async with EXPR as VAR:
    BLOCK
```

等同于：
```python
mgr = (EXPR)
aexit = type(mgr).__aexit__
aenter = type(mgr).__aenter__(mgr)
exc = True

VAR = await aenter
try:
    BLOCK
except:
    if not await aexit(mgr, *sys.exc_info()):
        raise
else:
    await aexit(mgr, None, None, None)
```

示例：

```python
async def commit(session, data):
    ...
    async with session.transaction():
        ...
        await session.update(data)
        ...
```

再比如：
```python
async with lock:
    ...
```

而不是：
```python
with (yield from lock):
    ...
```


### 异步迭代器和 `async for`

要支持异步迭代，需要：
- 对象必须实现一个`__aiter__`方法，返回一个异步迭代器对象买这个异步迭代器对象在每次迭代时会返回一个awaitable；
- 一个异步迭代器必须实现一个`__anext__`方法，在每次迭代时会返回一个awaitable；
- 要停止迭代，`__anext__`必须抛出一个`StopAsyncIteration`异常。

示例：

```python
class AsyncIterable:
    async def __aiter__(self):
        return self
    
    async def __anext__(self):
        data = await self.fetch_data()
        if data:
            return data
        else:
            raise StopAsyncIteration
    
    async def fetch_data(self):
        ...
```

新语法：
```python
async for TARGET in ITER:
    BLOCK
else:
    BLOCK2
```
语义上等于：
```python
iter = (ITER)
iter = await type(iter).__aiter__(iter)
running = True
while running:
    try:
        TARGET = await type(iter).__anext__(iter)
    except StopAsyncIteration:
        running = False
    else:
        BLOCK
else:
    BLOCK2
```